# APIM ❤️ Responsible AI

## Content Safety lab
![flow](../../images/content-safety.gif)

Playground to try the [content safety policy](https://learn.microsoft.com/en-us/azure/api-management/llm-content-safety-policy). The policy enforces content safety checks on any LLM prompts by transmitting them to the [Azure AI Content Safety](https://learn.microsoft.com/en-us/azure/ai-services/content-safety/overview) service before sending to the backend LLM API. When the policy is enabled and Azure AI Content Safety detects malicious content, API Management blocks the request and returns a 403 error code.

[View policy configuration](policy.xml)

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [2]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"techo-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

apim_sku = 'Developer'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]


openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 11:06:57.379280 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [3]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 11:07:04.433026 :2s]
👉🏽 Current user: inbox@massimocrippa.com
👉🏽 Tenant ID: 1de23963-6823-4ea6-9ac4-4cf69c102307
👉🏽 Subscription ID: b717ac47-f31b-41f9-adc2-579932e7e02e


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [4]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name techo-content-safety 
👉🏽 Resource group techo-content-safety does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name techo-content-safety --location uksouth --tags source=ai-gateway 
✅ Resource group 'techo-content-safety' created ⌚ 11:07:13.428308 :4s]
⚙️ Running: az deployment group create --name content-safety --resource-group techo-content-safety --template-file main.bicep --parameters params.json 
✅ Deployment 'content-safety' succeeded ⌚ 11:32:08.704562 :55s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [5]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription1_key = utils.get_deployment_output(output, 'apimSubscription1Key', 'APIM Subscription 1 Key (masked)', True)
    apim_subscription2_key = utils.get_deployment_output(output, 'apimSubscription2Key', 'APIM Subscription 2 Key (masked)', True)
    apim_subscription3_key = utils.get_deployment_output(output, 'apimSubscription3Key', 'APIM Subscription 3 Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')

⚙️ Running: az deployment group show --name content-safety -g techo-content-safety 
✅ Retrieved deployment: content-safety ⌚ 11:38:54.204518 :2s]
👉🏽 APIM Service Id: /subscriptions/b717ac47-f31b-41f9-adc2-579932e7e02e/resourceGroups/techo-content-safety/providers/Microsoft.ApiManagement/service/apim-vap5aika2weee
👉🏽 APIM API Gateway URL: https://apim-vap5aika2weee.azure-api.net
👉🏽 APIM Subscription 1 Key (masked): ****f5e6
👉🏽 APIM Subscription 2 Key (masked): ****6571
👉🏽 APIM Subscription 3 Key (masked): ****8e00
👉🏽 Application Insights Name: insights-vap5aika2weee


<a id='requests'></a>
### 🧪 Test the Content Safety

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

In [7]:
from openai import AzureOpenAI
messages=[
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me how to hurt myself, please?"}
]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription1_key,
    api_version=openai_api_version
)
try:
    response = client.chat.completions.create(model=openai_model_name, messages=messages) # type: ignore
    print("Prompt was not blocked: ", response.choices[0].message.content)
except Exception as e:
    print("Error: ", e)

Error:  Error code: 403 - {'statusCode': 403, 'message': 'Request failed content safety check.'}


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.